In [1]:
# some useful mysklearn package import statements and reloads
import importlib

import pandas as pd
import numpy as np
import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils

import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

# uncomment once you paste your myclassifiers.py into mysklearn package
import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MyKNeighborsClassifier, MyDummyClassifier, MyNaiveBayesClassifier, MyDecisionTreeClassifier, MyRandomForestClassifier

import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as myevaluation

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder


# ✈️ Flight Delay Classification ✈️

In [2]:
import pandas as pd
from sklearn.model_selection import KFold

# Load data
data = pd.read_csv('normalized_flights.csv')

# Compute flight delay (in minutes, assuming sched_dep_time and dep_time are timestamps or integers)
data['flight_delay'] = data['sched_dep_time'] - data['dep_time']

# Extract meaningful features for predicting delays
X = data[[
    'year', 'month', 'day', 'hour', 'minute',  # Time-based features
    'carrier', 'flight', 'tailnum',            # Operational features
    'origin', 'dest', 'distance', 'air_time', # Location & flight characteristics
    'dep_delay', 'sched_dep_time'             # Existing delay and timing
]]

# Target variable: flight delay
y = data['flight_delay']

# Initialize K-Fold cross-validation (10 splits)
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Placeholder for performance metrics
accuracies, error_rates, precisions, recalls, f1_scores, confusion_matrices = [], [], [], [], [], []


In [3]:
import numpy as np
import pandas as pd

# Load the dataset (assuming `data` is a pandas DataFrame containing the provided sample data)
data = pd.read_csv('normalized_flights.csv') 

# Define delay categories
def categorize_delay(delay):
    if delay <= 0:
        return 0
    elif 0 < delay <= 30:
        return 1
    elif 30 < delay <= 60:
        return 2
    elif 60 < delay <= 120:
        return 3
    elif 120 < delay <= 180:
        return 4
    else:
        return 5

# Map delays to categories
data['delay_category'] = data['arr_delay'].apply(categorize_delay)

# Convert categorical features to integers (label encoding)
categorical_cols = ['carrier', 'flight', 'tailnum', 'origin', 'dest']  # List your categorical columns
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col].astype(str))  # Convert to string if necessary and apply label encoding
    label_encoders[col] = le  # Store the encoder to use it later if needed

# Prepare features (X) and labels (y)
X = data[['dep_time', 'sched_dep_time', 'dep_delay', 'arr_time', 'sched_arr_time', 'arr_delay', 'air_time', 'distance', 'dest',
          'carrier', 'month', 'hour']].fillna(0)

y = data['delay_category'].values

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


### kNN Performance Evaluation & Confusion Matrix

In [269]:
import numpy as np
import pandas as pd

X = np.array(X) 
y = np.array(y) 

# Initialize StratifiedKFold with k=10 (assuming myevaluation provides this utility)
kf = myevaluation.stratified_kfold_split(X, y, n_splits=10, random_state=None, shuffle=False)

# Initialize lists to store metrics (accuracy, precision, recall, f1, confusion matrices, etc.)
knn_accuracies, knn_precisions, knn_recalls, knn_f1s, knn_conf_matrices = [], [], [], [], []

# Initialize lists to gather true and predicted values across all folds
all_y_true = []
all_y_pred = []

# Define the correct labels for the confusion matrix and metrics
labels = [0, 1, 2, 3, 4, 5]

# Loop over each fold
for train_index, test_index in kf:
    knn_X_train, knn_X_test = X[train_index], X[test_index]
    knn_y_train, knn_y_test = y[train_index], y[test_index]

    # Initialize and fit MyKNeighborsClassifier
    knn_model = MyKNeighborsClassifier()
    knn_model.fit(knn_X_train.tolist(), knn_y_train.tolist())
    
    # Make predictions on the test set
    knn_y_pred = knn_model.predict(knn_X_test.tolist())
    
    # Calculate and store metrics for this fold
    accuracy = myevaluation.accuracy_score(knn_y_test, knn_y_pred)
    precision_score = myevaluation.binary_precision_score(knn_y_test, knn_y_pred, labels= labels)  # Multi-class precision
    recall_score = myevaluation.binary_recall_score(knn_y_test, knn_y_pred, labels= labels)  # Multi-class recall
    f1 = myevaluation.binary_f1_score(knn_y_test, knn_y_pred, labels= labels)  # Multi-class F1 score
    confusion_matrix = myevaluation.confusion_matrix(knn_y_test, knn_y_pred, labels=labels)

    # Store fold metrics
    knn_accuracies.append(accuracy)
    knn_precisions.append(precision_score)
    knn_recalls.append(recall_score)
    knn_f1s.append(f1)
    knn_conf_matrices.append(confusion_matrix)
    
    # Collect all true and predicted labels for final confusion matrix
    all_y_true.extend(knn_y_test)
    all_y_pred.extend(knn_y_pred)

# Calculate the final confusion matrix using all folds' predictions
final_confusion_matrix = myevaluation.confusion_matrix(all_y_true, all_y_pred, labels=labels)
matrix_with_totals = myutils.calculate_confusion_matrix_totals(final_confusion_matrix)

In [270]:
print("===========================================")
print("kNN Classifier Performance")
print("===========================================")

# Calculate and print average performance
print(f"Accuracy: {np.mean(knn_accuracies):.2f}")
print(f"Error: {1 - np.mean(knn_accuracies):.2f}")
print(f"Precision: {np.mean(knn_precisions):.2f}")
print(f"Recall: {np.mean(knn_recalls):.2f}")
print(f"F1 Score: {np.mean(knn_f1s):.2f}\n")

# Display the confusion matrix
print("Confusion Matrix:")
myutils.display_confusion_matrix(matrix_with_totals, labels)

kNN Classifier Performance
Accuracy: 0.57
Error: 0.43
Precision: 0.60
Recall: 0.72
F1 Score: 0.64

Confusion Matrix:
  Survived    0    1    2    3    4    5    Total    Recognition %
----------  ---  ---  ---  ---  ---  ---  -------  ---------------
         0  264   98    4    0    0    3      369            71.54
         1  142  123   40    8    0    0      313            39.30
         2   37   88   73   29    0    0      227            32.16
         3    7   37   55  179   46    0      324            55.25
         4    2    1   14   71  175   21      284            61.62
         5    0    2    3   10   62  206      283            72.79


### Naive Bayes Evaluation & Confusion Matrix

In [4]:
import numpy as np
import pandas as pd

X = np.array(X) 
y = np.array(y) 

# Initialize StratifiedKFold with k=10
kf = myevaluation.stratified_kfold_split(X, y, n_splits=10, random_state=None, shuffle=False)

# Initialize lists to store metrics
nb_accuracies, nb_precisions, nb_recalls, nb_f1s, nb_conf_matrices = [], [], [], [], []

# Initialize lists to collect all true and predicted labels across all folds
all_y_true = []
all_y_pred = []

# Define the correct labels for the confusion matrix and metrics
labels = [0, 1, 2, 3, 4, 5]

# Loop over each fold
# Loop over each fold in Stratified K-Fold
for train_index, test_index in kf:
    nb_X_train, nb_X_test = X[train_index], X[test_index]
    nb_y_train, nb_y_test = y[train_index], y[test_index]

    # Initialize and fit MyNaiveBayesClassifier
    nb_model = MyNaiveBayesClassifier()
    nb_model.fit(nb_X_train.tolist(), nb_y_train.tolist())
    
    # Make predictions on the test set
    nb_y_pred = nb_model.predict(nb_X_test.tolist())
    
    # Calculate and store metrics for this fold
    accuracy = myevaluation.accuracy_score(nb_y_test, nb_y_pred)
    error = 1 - accuracy
    precision_score = myevaluation.binary_precision_score(nb_y_test, nb_y_pred, labels=labels)
    recall_score = myevaluation.binary_recall_score(nb_y_test, nb_y_pred, labels=labels)
    f1 = myevaluation.binary_f1_score(nb_y_test, nb_y_pred, labels=labels)
    confusion_matrix = myevaluation.confusion_matrix(nb_y_test, nb_y_pred, labels=labels)

    # Store metrics for this fold
    nb_accuracies.append(accuracy)
    nb_precisions.append(precision_score)
    nb_recalls.append(recall_score)
    nb_f1s.append(f1)
    nb_conf_matrices.append(confusion_matrix)

    # Collect all true and predicted labels for final confusion matrix
    all_y_true.extend(nb_y_test)
    all_y_pred.extend(nb_y_pred)
   

# Calculate the final confusion matrix using all folds' predictions
final_confusion_matrix = myevaluation.confusion_matrix(all_y_true, all_y_pred, labels=labels)
matrix_with_totals = myutils.calculate_confusion_matrix_totals(final_confusion_matrix)

In [272]:
print("===========================================")
print("Naive Bayes Classifier Performance")
print("===========================================")

# Calculate and print average performance
print(f"Accuracy: {np.mean(nb_accuracies):.2f}")
print(f"Error: {1 - np.mean(nb_accuracies):.2f}")
print(f"Precision: {np.mean(nb_precisions):.2f}")
print(f"Recall: {np.mean(nb_recalls):.2f}")
print(f"F1 Score: {np.mean(nb_f1s):.2f}\n")

# Display the confusion matrix
print("Confusion Matrix:")
myutils.display_confusion_matrix(matrix_with_totals, labels)

Naive Bayes Classifier Performance
Accuracy: 0.65
Error: 0.35
Precision: 0.71
Recall: 0.78
F1 Score: 0.74

Confusion Matrix:
  Survived    0    1    2    3    4    5    Total    Recognition %
----------  ---  ---  ---  ---  ---  ---  -------  ---------------
         0  289   46    9   13    8    4      369            78.32
         1   52  207   23   21    4    6      313            66.13
         2   18   36  117   39    8    9      227            51.54
         3   17   17   18  227   28   17      324            70.06
         4   13   12   12   36  181   30      284            63.73
         5   29   14   21   30   38  151      283            53.36


### Random Forest Performance Evaluation & Confusion Matrix

In [5]:
import numpy as np
import pandas as pd

# Convert data to numpy arrays if not already
X = np.array(X) 
y = np.array(y) 

# Initialize StratifiedKFold with k=10 (assuming myevaluation provides this utility)
kf = myevaluation.stratified_kfold_split(X, y, n_splits=10, random_state=None, shuffle=False)

# Initialize lists to store metrics (accuracy, precision, recall, f1, confusion matrices, etc.)
rf_accuracies, rf_precisions, rf_recalls, rf_f1s, rf_conf_matrices = [], [], [], [], []

# Initialize lists to gather true and predicted values across all folds
all_y_true = []
all_y_pred = []

# Define the correct labels for the confusion matrix and metrics
labels = [0, 1, 2, 3, 4, 5]

# Loop over each fold
for train_index, test_index in kf:
    rf_X_train, rf_X_test = X[train_index], X[test_index]
    rf_y_train, rf_y_test = y[train_index], y[test_index]

    # Initialize and fit RandomForestClassifier
    rf_model = MyRandomForestClassifier(n=10, m=4, f=2)  # Adjust hyperparameters as needed
    rf_model.fit(rf_X_train, rf_y_train)
    
    # Make predictions on the test set
    rf_y_pred = rf_model.predict(rf_X_test)

    print(f"Actual: {rf_y_test}")
    print(f"Predicted: {rf_y_pred}")
    
    # Calculate and store metrics for this fold
    accuracy = myevaluation.accuracy_score(rf_y_test, rf_y_pred)
    precision_score = myevaluation.binary_precision_score(rf_y_test, rf_y_pred, labels=labels)  # Multi-class precision
    recall_score = myevaluation.binary_recall_score(rf_y_test, rf_y_pred, labels=labels)  # Multi-class recall
    f1 = myevaluation.binary_f1_score(rf_y_test, rf_y_pred, labels=labels)  # Multi-class F1 score
    confusion_matrix = myevaluation.confusion_matrix(rf_y_test, rf_y_pred, labels=labels)

    # Store fold metrics
    rf_accuracies.append(accuracy)
    rf_precisions.append(precision_score)
    rf_recalls.append(recall_score)
    rf_f1s.append(f1)
    rf_conf_matrices.append(confusion_matrix)
    
    # Collect all true and predicted labels for final confusion matrix
    all_y_true.extend(rf_y_test)
    all_y_pred.extend(rf_y_pred)

# Calculate the final confusion matrix using all folds' predictions
final_confusion_matrix = myevaluation.confusion_matrix(all_y_true, all_y_pred, labels=labels)
matrix_with_totals = myutils.calculate_confusion_matrix_totals(final_confusion_matrix)


num trees: 4
[['Leaf', 21.0, 1080, 1080], ['Attribute', 'att11', ['Value', 5.0, ['Leaf', 5.0, 2, 1080]], ['Value', 6.0, ['Leaf', 6.0, 16, 1080]], ['Value', 7.0, ['Leaf', 7.0, 26, 1080]], ['Value', 8.0, ['Leaf', 8.0, 31, 1080]], ['Value', 9.0, ['Leaf', 9.0, 33, 1080]], ['Value', 10.0, ['Leaf', 10.0, 44, 1080]], ['Value', 11.0, ['Leaf', 11.0, 37, 1080]], ['Value', 12.0, ['Leaf', 12.0, 37, 1080]], ['Value', 13.0, ['Leaf', 13.0, 29, 1080]], ['Value', 14.0, ['Leaf', 14.0, 51, 1080]], ['Value', 15.0, ['Leaf', 15.0, 47, 1080]], ['Value', 16.0, ['Leaf', 16.0, 76, 1080]], ['Value', 17.0, ['Leaf', 17.0, 81, 1080]], ['Value', 18.0, ['Leaf', 18.0, 54, 1080]], ['Value', 19.0, ['Leaf', 19.0, 117, 1080]], ['Value', 20.0, ['Leaf', 20.0, 83, 1080]], ['Value', 21.0, ['Leaf', 21.0, 80, 1080]], ['Value', 22.0, ['Leaf', 22.0, 65, 1080]], ['Value', 23.0, ['Leaf', 23.0, 69, 1080]]], ['Leaf', 21.0, 1080, 1080], ['Leaf', 21.0, 1080, 1080]]
Actual: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

KeyError: 21.0

In [274]:
print("===========================================")
print("Random Forest Classifier Performance")
print("===========================================")

# Calculate and print average performance
print(f"Accuracy: {np.mean(rf_accuracies):.2f}")
print(f"Error: {1 - np.mean(rf_accuracies):.2f}")
print(f"Precision: {np.mean(rf_precisions):.2f}")
print(f"Recall: {np.mean(rf_recalls):.2f}")
print(f"F1 Score: {np.mean(rf_f1s):.2f}\n")

# Display the confusion matrix
print("Confusion Matrix:")
myutils.display_confusion_matrix(matrix_with_totals, labels)


Random Forest Classifier Performance
Accuracy: 0.98
Error: 0.02
Precision: 0.99
Recall: 1.00
F1 Score: 0.99

Confusion Matrix:
  Survived    0    1    2    3    4    5    Total    Recognition %
----------  ---  ---  ---  ---  ---  ---  -------  ---------------
         0  369    0    0    0    0    0      369           100.00
         1    0  313    0    0    0    0      313           100.00
         2    0    4  222    1    0    0      227            97.80
         3    0    0   10  311    3    0      324            95.99
         4    0    0    0    5  278    1      284            97.89
         5    6    0    0    1    2  274      283            96.82
